<a href="https://colab.research.google.com/github/aytnihca/509-Project/blob/main/Achintya_Raya_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question 1

In [ ]:
import numpy as np
from scipy import stats
!pip install py_vollib_vectorized

In [ ]:
# Part a)
def GeoBMPaths2(S0, nu, sigma, T, Nsteps, Npaths, seed=None):
    s = sigma * (T / Nsteps) ** .5
    n = nu * T / Nsteps
    increments = np.zeros((Nsteps+1, Npaths))
    increments[0] = S0
    rng = np.random.default_rng(seed)
    increments[1:] = np.exp(n + rng.normal(scale=s, size=(Nsteps,Npaths)))
    Ssample = np.cumprod(increments, axis=0)
    return Ssample

def Q1a(S0,sigma,rho,K,T1,T2,Nsteps,Npaths,seed=None):
    paths = GeoBMPaths2(S0, rho-sigma**2/2, sigma, T2, Nsteps, Npaths, seed=12345)

    k = int(np.ceil(T1*Nsteps/T2))
    t_k = T2*k/Nsteps
    t_k_1 = T2*(k-1)/Nsteps
    S_t_k = paths[k,:]
    S_t_k_1 = paths[k-1,:]
    S_T1 = (T1 - t_k_1)/(t_k - t_k_1)*S_t_k + (t_k - T1)/(t_k - t_k_1)*S_t_k_1

    integral = (S_T1+S_t_k)*(t_k - T1)/2
    for j in range(k+1, Nsteps+1):
      S_t_j = paths[j,:]
      S_t_j_1 = paths[j-1,:]
      t_j = T2*j/Nsteps
      t_j_1 = T2*(j-1)/Nsteps
      integral+= (S_t_j + S_t_j_1)*(t_j - t_j_1)/2

    disc_payoff = np.exp(-rho * T2) * np.maximum(integral/(T2-T1) - K,0)
    price =  disc_payoff.mean()
    CI = stats.norm.interval(.95, loc=price, scale=stats.sem(disc_payoff))
    return price, CI

In [ ]:
# Part b)
from py_vollib_vectorized.implied_volatility import vectorized_implied_volatility
sigma = vectorized_implied_volatility(593.05, 16219.94, 16700, 3.5/12, 0.01, model='black_scholes', flag = 'c')['IV']
print('The implied volatility of the index is: ', sigma[0])

The implied volatility of the index is:  0.22326640281046714


In [ ]:
# Part c)
price, CI = Q1a(16219.94,sigma[0], 0.01,17000,1/12,4/12, 10000, 1000,seed=None)
print('The price of the asian call option is: ', price)
print('The 95% confidence interval is: ', CI)

The price of the asian call option is:  299.82002029816374
The 95% confidence interval is:  (259.3679616753818, 340.27207892094566)


Part d) No, it does not. If the option price falls outside the confidence intervals, it just means that we are in a situation that was unlikely to occur. This is obviously different from risk-free gain beyond the interest rate.

In [ ]:
# Part e)
def Antithetic_GeoBMPaths2(S0, nu, sigma, T, Nsteps, Npaths, seed=None):
    s = sigma * (T / Nsteps) ** .5
    n = nu * T / Nsteps
    increments_1 = np.zeros((Nsteps+1, Npaths))
    increments_1[0] = S0
    increments_2 = increments_1.copy()

    rng = np.random.default_rng(seed)
    sample1 = rng.normal(scale=s, size=(Nsteps,Npaths))
    sample2 = rng.normal(scale=s, size=(Nsteps,Npaths))

    increments_1[1:] = np.exp(n + sample1)
    increments_2[1:] = np.exp(n - sample2)

    Ssample1 = np.cumprod(increments_1, axis=0)
    Ssample2 = np.cumprod(increments_2, axis=0)
    return Ssample1, Ssample2

def Q1e(S0,sigma,rho,K,T1,T2,Nsteps,Npaths,seed=None):
    paths_1, paths_2 = Antithetic_GeoBMPaths2(S0, rho-sigma**2/2, sigma, T2, Nsteps, Npaths, seed=12345)

    k = int(np.ceil(T1*Nsteps/T2))
    t_k = T2*k/Nsteps
    t_k_1 = T2*(k-1)/Nsteps
    S1_t_k = paths_1[k,:]
    S1_t_k_1 = paths_1[k-1,:]
    S2_t_k = paths_2[k,:]
    S2_t_k_1 = paths_2[k-1,:]
    S1_T1 = (T1 - t_k_1)/(t_k - t_k_1)*S1_t_k + (t_k - T1)/(t_k - t_k_1)*S1_t_k_1
    S2_T1 = (T1 - t_k_1)/(t_k - t_k_1)*S2_t_k + (t_k - T1)/(t_k - t_k_1)*S2_t_k_1

    integral_1 = (S1_T1+S1_t_k)*(t_k - T1)/2
    integral_2 = (S2_T1+S2_t_k)*(t_k - T1)/2
    for j in range(k+1, Nsteps+1):
      S1_t_j = paths_1[j,:]
      S1_t_j_1 = paths_1[j-1,:]
      S2_t_j = paths_2[j,:]
      S2_t_j_1 = paths_2[j-1,:]

      t_j = T2*j/Nsteps
      t_j_1 = T2*(j-1)/Nsteps

      integral_1+= (S1_t_j + S1_t_j_1)*(t_j - t_j_1)/2
      integral_2+= (S2_t_j + S2_t_j_1)*(t_j - t_j_1)/2

    disc_payoff_1 = np.exp(-rho * T2) * np.maximum(integral_1/(T2-T1) - K,0)
    disc_payoff_2 = np.exp(-rho * T2) * np.maximum(integral_2/(T2-T1) - K,0)
    disc_payoff = (disc_payoff_1+disc_payoff_2)/2
    price =  disc_payoff.mean()
    CI = stats.norm.interval(.95, loc=price, scale=stats.sem(disc_payoff))
    return price, CI

price, CI = Q1e(16219.94,sigma[0], 0.01,17000,1/12,4/12, 10000, 1000,seed=None)
print('The price of the asian call option is: ', price)
print('The 95% confidence interval is: ', CI)

The price of the asian call option is:  308.27452914801745
The 95% confidence interval is:  (279.3384369515039, 337.210621344531)


Part e) Since we are using geometric brownian motion, it is easy to generate numbers which have negative correlation. Thus, the confidence interval is guaranteed to shrink. Antithetic sampling is guaranteed to reduce the variance by a factor 2. Therefore, the length of the confidence interval, which scales with the standard deviation, is reduced by $\sqrt{2}$ which is what we observe as well.
  

# Question 2

In [ ]:
# Part a)
def Q2a(alpha,beta,gamma,rho0,T,Nsteps,Npaths,seed=None):
    dt = T/Nsteps
    sqrt_dt = np.sqrt(dt)

    paths = np.zeros((Nsteps+1, Npaths))
    paths[0, :] = rho0

    rng = np.random.default_rng(seed)
    for i in range(1, Nsteps + 1):
        dW = rng.normal(scale=sqrt_dt, size=Npaths)
        paths[i,:] = paths[i-1,:] + alpha*(beta - paths[i-1,:])*dt + gamma*np.sqrt(np.maximum(paths[i-1,:],0))*dW
    return paths

In [ ]:
# Part b)

def Q2b(alpha,beta,gamma,rho0,T,Nsteps,Npaths,seed=None):
    paths = Q2a(alpha,beta,gamma,rho0,T,Nsteps,Npaths,seed=12345)
    T1=0; T2=T
    # Nearly copied the code from Q1a
    k = int(np.ceil(T1*Nsteps/T2))
    t_k = T2*k/Nsteps
    t_k_1 = T2*(k-1)/Nsteps
    S_t_k = paths[k,:]
    S_t_k_1 = paths[k-1,:]
    S_T1 = (T1 - t_k_1)/(t_k - t_k_1)*S_t_k + (t_k - T1)/(t_k - t_k_1)*S_t_k_1

    integral = (S_T1+S_t_k)*(t_k - T1)/2
    for j in range(k+1, Nsteps+1):
      S_t_j = paths[j,:]
      S_t_j_1 = paths[j-1,:]
      t_j = T2*j/Nsteps
      t_j_1 = T2*(j-1)/Nsteps
      integral+= (S_t_j + S_t_j_1)*(t_j - t_j_1)/2

    integral = np.exp(-1*integral)
    price =  integral.mean()
    CI = stats.norm.interval(.95, loc=price, scale=stats.sem(integral))
    return price, CI

In [ ]:
# Part c)
import pandas as pd

def Q2c(parameters):
  daily_returns = pd.read_csv('dataQ2A4.csv', index_col=0)
  daily_returns = daily_returns.values

  T = 30; Nsteps = 120; gamma = 0; Npaths = 1; rho0 = 0.05
  alpha, beta = parameters

  difference = 0
  for i in range(Nsteps):
    t = ((i+1)*T)/Nsteps
    difference= difference + (Q2a(alpha,beta,gamma,rho0,t,Nsteps,Npaths,seed=12345)[120] - daily_returns[i])**2
  return difference

In [ ]:
# Part d)
from scipy.optimize import minimize

initial_guess = [1, 0.02]
minimizers = minimize(Q2c, initial_guess).x
minimized_value = Q2c(minimizers)
print("Minimizers:", minimizers)
print("Corresponding value of Q2c:", minimized_value)


daily_returns = pd.read_csv('dataQ2A4.csv', index_col=0)
daily_returns = daily_returns.values
alpha, beta = minimizers
b = minimized_value
c = daily_returns.mean()
T = 30; Nsteps = 120; Npaths = 100; rho0 = 0.05
gamma = np.sqrt(b/Nsteps/c)

price, CI = Q2b(alpha,beta,gamma,0.05,30,120,100,seed=12345)
print('The price is: ', price)
print('The 95% confidence interval of the price is: ', CI)

Minimizers: [0.4745616  0.03288872]
Corresponding value of Q2c: [9.93538043e-05]
The price is:  0.36016921095361054
The 95% confidence interval of the price is:  (0.3595590757771815, 0.3607793461300396)
